In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import os
from fnmatch import fnmatch
from pathlib import Path

from src.imc24 import score

In [2]:
class CFG:
    dir_train = 'data\\train'
    path_target = 'data\\train\\train_labels.csv'

In [3]:
def get_validation_data(dataset):
    df_val = pd.read_csv(CFG.path_target)
    df_val = df_val.loc[df_val['dataset'] == dataset]
    df_val['image_path'] = f'data\\train\\{dataset}\\images\\' + df_val['image_name']
    return df_val[['image_path', 'dataset', 'scene', 'rotation_matrix', 'translation_vector']]

In [4]:
datasets = pd.read_csv(CFG.path_target)['dataset'].unique()
datasets = list(set(datasets) - set(['pond', 'lizard']))

results = []

for dataset in datasets:
    df_val = get_validation_data(dataset)
    for file in tqdm(os.listdir(CFG.dir_train)):
        if fnmatch(file, f'result_{dataset}*.csv'):
            # print(file)
            file_params = file.replace(f'result_{dataset}_', '').replace('.csv', '').split('_')

            params = []

            pairs_threshold, pairs_minnumber = file_params[0][5:].split('x')
            match_threshold = file_params[-1][8:]
            matcher = None
            extractor = None
            resize_to = 0

            if len(file_params) == 3: # LoFTR
                matcher = 'LoFTR'
                extractor = 'LoFTR'
                resize_to = file_params[1][5:]
            else:
                matcher = file_params[2]

                n_letters = sum(c.isalpha() for c in file_params[1])
                extractor = file_params[1][:n_letters]
                resize_to = file_params[1][n_letters:]

            params = [pairs_threshold, pairs_minnumber, extractor, matcher, resize_to, match_threshold]
            # print(params)

            df_preds = pd.read_csv(os.path.join(CFG.dir_train, file))
            sc = score(df_val, df_preds, verbose=False)
            results.append([dataset, sc, *params])

100%|██████████| 61/61 [19:15<00:00, 18.94s/it]   


In [5]:
df_results = pd.DataFrame(results, columns=['dataset', 'score', 'pairs_threshold', 'pairs_minnumber', 'extractor', 'matcher', 'resize_to', 'match_threshold'])
df_results.to_csv('results.csv', index=False)
df_results

,dataset,score,pairs_threshold,pairs_minnumber,extractor,matcher,resize_to,match_threshold
0,transp_obj_glass_cup,0.000000,0.1,20,LoFTR,LoFTR,512,0.4
1,transp_obj_glass_cup,0.025253,0.1,30,disk,LightGlue,1024,0.01
2,transp_obj_glass_cup,0.025253,0.1,30,disk,LightGlue,768,0.01
3,transp_obj_glass_cup,0.005051,0.1,30,disk,LightGlue,768,0.3
4,transp_obj_glass_cup,0.005051,0.1,30,doghardnet,LightGlue,1024,0.01
5,transp_obj_glass_cup,0.005051,0.1,30,LoFTR,LoFTR,512,0.3
6,transp_obj_glass_cup,0.035354,0.1,30,LoFTR,LoFTR,512,0.4
7,transp_obj_glass_cup,0.000000,0.1,30,sift,LightGlue,1024,0.01
8,transp_obj_glass_cup,0.000000,0.1,5,LoFTR,LoFTR,512,0.4
9,transp_obj_glass_cup,0.015152,0.3,5,aliked,LightGlue,1024,0.01


In [11]:
df_results.sort_values(['dataset', 'score'], ascending=False).groupby('dataset').head(2)

,dataset,score,pairs_threshold,pairs_minnumber,extractor,matcher,resize_to,match_threshold
29,transp_obj_glass_cylinder,0.005051,0.3,5,aliked,LightGlue,1024,0.01
21,transp_obj_glass_cylinder,0.000000,0.1,30,disk,LightGlue,1024,0.01
6,transp_obj_glass_cup,0.035354,0.1,30,LoFTR,LoFTR,512,0.4
1,transp_obj_glass_cup,0.025253,0.1,30,disk,LightGlue,1024,0.01
35,multi-temporal-temple-baalshamin,0.241026,0.1,30,disk,LightGlue,768,0.3
33,multi-temporal-temple-baalshamin,0.235897,0.1,30,disk,LightGlue,1024,0.01
13,dioscuri,0.405473,0.1,30,disk,LightGlue,768,0.3
11,dioscuri,0.398010,0.1,30,disk,LightGlue,1024,0.01
44,church,0.285047,0.1,30,disk,LightGlue,1024,0.01
45,church,0.285047,0.1,30,disk,LightGlue,768,0.01
